In [240]:
# Importing the packages

from pycatia import catia
from pycatia.mec_mod_interfaces.part_document import PartDocument
import time

In [241]:
# Making the Catia instance

caa = catia()
application = caa.application
documents = application.documents

# Closing if any document is open
if documents.count > 0:
    for document in documents:
        document.close()
        
# Create Part
documents.add('Part')

# Get references to document, parts and tools to create geometries
document: PartDocument = application.active_document        # Get active document
part = document.part                                        # Get Part
partbody = part.bodies[0]                                   # Get default PartBody by index
# partbody = part.bodies.item("PartBody")                   # Get default PartBody by name
sketches = partbody.sketches                                # Get sketches on PartBody 
hybrid_bodies = part.hybrid_bodies                          # Get Hybrid Bodies 
hsf = part.hybrid_shape_factory                             # Get Hybrid Shape Factory 
shpfac = part.shape_factory                                 # Get Shape Factory 
selection = document.selection                              # Get document selection

In [242]:
# Get references to main planes

plane_XY = part.origin_elements.plane_xy
plane_YZ = part.origin_elements.plane_yz
plane_ZX = part.origin_elements.plane_zx

In [243]:
# Create a new HybridBody to hold the construction elements

construction_elements = hybrid_bodies.add()
construction_elements.name = "construction_elements"

# Create a global reference point
p_0 = hsf.add_new_point_coord(0, 0, 0)
p_0.name = "p_0"
construction_elements.append_hybrid_shape(p_0)

# Create a point in Z axis
p_z = hsf.add_new_point_coord(0, 0, 1)
p_z.name = "p_z"
construction_elements.append_hybrid_shape(p_z)

# Z dir
z_dir = hsf.add_new_line_pt_pt(p_0,p_z)
z_dir.name = "z_dir"
construction_elements.append_hybrid_shape(z_dir)

In [244]:
# Set PartBody as a working object
part.in_work_object = partbody

# Shaft radius
shaft_radius = 1

# Create sketch in plane_XY
sketch_1 = sketches.add(plane_XY)
sketch_1.name = "Shaft_sketch"

# Sketch open edition
ske2D_1 = sketch_1.open_edition()

circle1 = ske2D_1.create_closed_circle(0, 0, shaft_radius)

# sketch close edition
sketch_1.close_edition()

# Adding pad to the sketch to make a 3D part
shaft_top = shpfac.add_new_pad(sketch_1, 10)
shaft_top.name = "Shaft_top"

# Update the document
document.part.update()

In [245]:
# Add blades

sketch_2 = sketches.add(plane_ZX)
sketch_2.name = "Blade_sketch"
ske2D_2 = sketch_2.open_edition()

# Blade points
s2p1 = (6, -1)
s2p2 = (7.5, 1.5)
s2p3 = (8, 1)
s2p4 = (6.5, -1.5)

ske2D_2.create_line(*s2p1,*s2p2)
ske2D_2.create_line(*s2p2,*s2p3)
ske2D_2.create_line(*s2p3,*s2p4)
ske2D_2.create_line(*s2p4,*s2p1)

sketch_2.close_edition()

blade = shpfac.add_new_pad(sketch_2, 56.4)
blade.name = "Single_blade"

document.part.update()

In [246]:
# Circular pattern for blades
# Create an angular pattern around Z axis

# add_new_rect_pattern arguments:
# i_shape_to_copy : blade
# i_nb_of_copies_in_radial_dir : 1
# i_nb_of_copies_in_angular_dir : 5
# i_step_in_radial_dir : 0 mm
# i_step_in_angular_dir : 360/5 deg
# i_shape_to_copy_position_along_radial_dir : 1 (to leave a space between patterns)
# i_shape_to_copy_position_along_angular_dir : 1 (to leave a space between patterns)
# i_rotation_center : z_dir
# i_rotation_axis : z_dir
# i_is_reversed_rotation_axis : True
# i_rotation_angle : 0
# i_is_radius_aligned : True

# Create circular pattern around Z axis
blades = shpfac.add_new_circ_pattern(blade, 1, 5, 0, 360/5, 1, 1, z_dir, z_dir, True, 0, True)
blades.name = "Blades"

document.part.update()

In [247]:
# Making the body

#Sketching spline for the body
sketch_3 = sketches.add(plane_ZX)
sketch_3.name = "Body_sketch"

skbody = sketch_3.open_edition()

s3p1 = skbody.create_control_point(-10, 28)
s3p2 = skbody.create_control_point(-6, 26)
s3p3 = skbody.create_control_point(0, 0)
s3p4 = skbody.create_control_point(-10, -35)
s3_points = [s3p1, s3p2, s3p3, s3p4]

spline = skbody.create_spline(s3_points)
mid_line = skbody.create_line(-10, 28, -10, -35)

sketch_3.close_edition()

# Create a global reference point
ra_p1 = hsf.add_new_point_coord(20, 0, -10)
ra_p1.name = "p1"
construction_elements.append_hybrid_shape(ra_p1)

# Create a global reference point
ra_p2 = hsf.add_new_point_coord(-30, 0, -10)
ra_p2.name = "p2"
construction_elements.append_hybrid_shape(ra_p2)

# Add revolution axis
rev_axis = hsf.add_new_line_pt_pt(ra_p1,ra_p2)
rev_axis.name = "body_revolution_axis"
construction_elements.append_hybrid_shape(rev_axis)

document.part.update()

# Making the shaft and updating document
main_body = shpfac.add_new_shaft(sketch_3)
main_body.revolute_axis = rev_axis
main_body.name = "Main_Body"

document.part.update()

In [248]:
# Creating the tail

# Making new reference plane for sketching ellipse for the tail profile
plane_ref = hsf.add_new_plane_offset(plane_YZ, -25, False)
construction_elements.append_hybrid_shape(plane_ref)
plane_ref.name = "Tail_sketch_plane"

document.part.update()

sketch_4 = sketches.add(plane_ref)
sketch_4.name = "Tail_sketch"

tail = sketch_4.open_edition()
ellipse = tail.create_closed_ellipse(0, -10, 0, 0, 1.5, 3)
sketch_4.close_edition()

tail_pad = shpfac.add_new_pad(sketch_4, -50)
tail_pad.name = "Tail"

document.part.update()

In [249]:
# Producing the horizontal stabilizer

sketch_5 = sketches.add(plane_ZX)
sketch_5.name = "Horizontal_stabilizer_sketch"
stabilizer = sketch_5.open_edition()

s5p1 = (-7, -60)
s5p2 = (-7, -65)
s5p3 = (-6, -65)
s5p4 = (-6, -60)

stabilizer.create_line(*s5p1, *s5p2)
stabilizer.create_line(*s5p2, *s5p3)
stabilizer.create_line(*s5p3, *s5p4)
stabilizer.create_line(*s5p4, *s5p1)

sketch_4.close_edition()

hstabilizer = shpfac.add_new_pad(sketch_5, 10)
hstabilizer.name = "Horizontal_stabilizer"
hstabilizer_2 = shpfac.add_new_pad(sketch_5, -10)
hstabilizer_2.name = "Horizontal_stabilizer"

document.part.update()

In [250]:
# Making the vertical Stabilizer

sketch_6 = sketches.add(plane_ZX)
sketch_6.name = "Vertical_stabilizer_sketch"
stabilizer = sketch_6.open_edition()

s6p1 = (-13, -72)
s6p2 = (-7, -72)
s6p3 = (2, -80)
s6p4 = (2, -85)
s6p5 = (-13, -82)

stabilizer.create_line(*s6p1, *s6p2)
stabilizer.create_line(*s6p2, *s6p3)
stabilizer.create_line(*s6p3, *s6p4)
stabilizer.create_line(*s6p4, *s6p5)
stabilizer.create_line(*s6p5, *s6p1)


sketch_6.close_edition()

vstabilizer = shpfac.add_new_pad(sketch_6, 1.5)
vstabilizer.name = "Vertical_stabilizer"
vstabilizer_2 = shpfac.add_new_pad(sketch_6, -1.5)
vstabilizer_2.name = "Vertical_stabilizer"

document.part.update()

In [251]:
# Rear fan shaft

rear_fan_radius = 0.5

sketch_7 = sketches.add(plane_ZX)
sketch_7.name = "Shaft_rear_sketch"

r_shaft = sketch_7.open_edition()
circle1 = r_shaft.create_closed_circle(0, -82.5, rear_fan_radius)
sketch_7.close_edition()

shaft_rear = shpfac.add_new_pad(sketch_7, 4)
shaft_rear.name = "Shaft_rear"

document.part.update()

In [252]:
# Add rear blades

sketch_8 = sketches.add(plane_XY)
sketch_8.name = "Blade_rear_sketch"
r_blade_sketch = sketch_8.open_edition()

s8p1 = (-82.1, 3)
s8p2 = (-82.2, 3.3)
s8p3 = (-82.9, 3)
s8p4 = (-82.8, 2.8)

r_blade_sketch.create_line(*s8p1,*s8p2)
r_blade_sketch.create_line(*s8p2,*s8p3)
r_blade_sketch.create_line(*s8p3,*s8p4)
r_blade_sketch.create_line(*s8p4,*s8p1)

sketch_7.close_edition()

blade_rear = shpfac.add_new_pad(sketch_8, 8)
blade_rear.name = "Single_blade_rear"

document.part.update()

In [253]:
# Create an angular pattern around shaft axis

p_rear_center = hsf.add_new_point_coord(-82.5, 0, 0)
p_rear_center.name = "p_rear_center"
construction_elements.append_hybrid_shape(p_rear_center)

p_y = hsf.add_new_point_coord(-82.5, 1, 0)
p_y.name = "p_y"
construction_elements.append_hybrid_shape(p_y)

shaft_dir = hsf.add_new_line_pt_pt(p_rear_center,p_y)
shaft_dir.name = "shaft_dir"
construction_elements.append_hybrid_shape(shaft_dir)

document.part.update()

# add_new_rect_pattern arguments:
# i_shape_to_copy : blade
# i_nb_of_copies_in_radial_dir : 1
# i_nb_of_copies_in_angular_dir : 4
# i_step_in_radial_dir : 0 mm
# i_step_in_angular_dir : 360/4 deg
# i_shape_to_copy_position_along_radial_dir : 1 (to leave a space between patterns)
# i_shape_to_copy_position_along_angular_dir : 1 (to leave a space between patterns)
# i_rotation_center : shaft_dir
# i_rotation_axis : shaft_dir
# i_is_reversed_rotation_axis : True
# i_rotation_angle : 0
# i_is_radius_aligned : True

# Create circular pattern around Z axis
blades_rear = shpfac.add_new_circ_pattern(blade_rear, 1, 4, 0, 90, 1, 1, shaft_dir, shaft_dir, True, 0, True)
blades_rear.name = "Blades_rear"

document.part.update()

In [254]:
# Landing gear

sketch_9 = sketches.add(plane_ZX)
sketch_9.name = "Landing_gear_sketch"

landing_gear_sketch = sketch_9.open_edition()

landing_gear_radius = 1
landing_gear_sketch.create_closed_circle(-10, -3, landing_gear_radius)

sketch_9.close_edition()

document.part.update()

# Create a curve
landing_gear_path = hsf.add_new_spline()
landing_gear_path.name = "Landing_gear_path"
landing_gear_path.add_point(hsf.add_new_point_coord(-3, 0, -10))
landing_gear_path.add_point(hsf.add_new_point_coord(-10, 8, -15))
landing_gear_path.add_point(hsf.add_new_point_coord(-15, 15, -25))
landing_gear_path.add_point(hsf.add_new_point_coord(0, 15, -25))
landing_gear_path.add_point(hsf.add_new_point_coord(15, 15, -25))
landing_gear_path.add_point(hsf.add_new_point_coord(10, 8, -15))
landing_gear_path.add_point(hsf.add_new_point_coord(3, 0, -10))

# Append guide splines 
construction_elements.append_hybrid_shape(landing_gear_path)
    
document.part.update()

In [255]:
# Create a rib

# Reference to sketch
ref_sketch_9 = part.create_reference_from_object(sketch_9) 
rib1 = shpfac.add_new_rib_from_ref(ref_sketch_9, landing_gear_path)
rib1.name = "Landing_gear"

document.part.update()

In [256]:
# Getting the landing gear for the other side

landing_gears = shpfac.add_new_circ_pattern(rib1, 1, 2, 0, 180, 1, 1, z_dir, z_dir, True, 0, True)
landing_gears.name = "Landing_gears"

document.part.update()

In [257]:
# Zoom in and save the document

active_viewer = caa.active_window.active_viewer
active_viewer.zoom_in()

# Hiding the construction elements
selection.clear()
selection.add(construction_elements.hybrid_shapes)
selection.add(plane_XY)
selection.add(plane_YZ)
selection.add(plane_ZX)
selection.vis_properties.set_show(1)    # 0 for show, 1 for hide
selection.clear()

# Saving the file to specified path 
document.save_as(r"C:\Project\Python\Product\Helicopter.CATpart", overwrite= True)

In [258]:
# Rotating the blade and shaft animation

# Show the magic
# Rotate the body and blades
for i in range(36):
    shpfac.add_new_rotate2(z_dir, 10)
    time.sleep(0.25)
    document.part.update()

while True:
    part.in_work_object = blades
    shpfac.add_new_rotate2(z_dir, 10)
    part.in_work_object = partbody
    time.sleep(0.25)
    document.part.update()

KeyboardInterrupt: 